In [1]:
!pip install pandas

import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize
from datetime import datetime, date, timedelta
import os.path 
from os import path

     |████████████████████████████████| 10.4MB 3.5MB/s eta 0:00:01    |█████████████████████████████▌  | 9.6MB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 20.1MB 154kB/s  eta 0:00:01    |████████▊                       | 5.5MB 30.0MB/s eta 0:00:01
     |████████████████████████████████| 512kB 16.9MB/s eta 0:00:01


In [4]:
#returns weather json data from weatherstack api
def request_weather_json(key, date, location):
    params = {
        'access_key': key,
        'query': location,
        'historical_date': date,
        'hourly': '1',
        'interval': '1'
        }

    api_result = requests.get('https://api.weatherstack.com/historical', params)
    api_response = api_result.json()
    #print(api_response['location'])
    return api_response

In [5]:
#Formats weather json data and outputs to detailed_weather_report.csv
def append_weather_data(json):
    json_historical = json_normalize(data=json['historical'][date], errors='ignore')
    detailed_weather_report = json_normalize(data=json_historical['hourly'][0], meta=['day'], meta_prefix='pre', errors='ignore')
    detailed_weather_report['location'] = json['location']['name']
    detailed_weather_report['date'] = date
    
    if path.exists("detailed_weather_report.csv"):
        detailed_weather_report.to_csv('detailed_weather_report.csv', mode='a', header=False)
    else:
        detailed_weather_report.to_csv('detailed_weather_report.csv')

In [6]:
def create_report():
    detailed_weather_df = pd.read_csv("detailed_weather_report.csv")
    weather_df = detailed_weather_df[detailed_weather_df['Unnamed: 0'].between(9,17)]
    weather_df['precip'].replace({0:np.nan}, inplace=True)
    weather_df = weather_df.groupby(['location','date']).agg({'precip':'count', 'temperature': 'max'}).reset_index().rename(columns={'precip':'Hours Rain', 'temperature':'Max Temp'})
    weather_df.to_csv('weather_report.csv')
    weather_df

In [7]:
today = datetime.now().date()
week_start = today - timedelta(days=today.weekday()+1)

# this will give you a list containing all of the dates
week_days = [(week_start + timedelta(days=x)).isoformat() for x in range((today-week_start).days + 1)]

week_days

['2020-01-19', '2020-01-20', '2020-01-21']

In [ ]:
#regions_dict = {'RegionID':1, 3, 6, 8 'Region':'Glasgow, UK', 'leeming, UK', 'London, UK', 'Belfast'}
regions_dict = {('RegionID':1,'Region':'Glasgow, UK')}
print(regions_dict)

In [62]:
key = '8c5dad538cf97d8347dbfda0e9369f25'
#dates = ['2020-01-01', '2020-01-02']
dates = week_days
regions = ['Glasgow, UK', 'leeming, UK', 'London, UK', 'Belfast']
regions_dict = {'RegionID':1, 3, 6, 8 'Region':'Glasgow, UK', 'leeming, UK', 'London, UK', 'Belfast'}

for location in locations:
    for date in dates:    
        json = request_weather_json(key, date, regions)
        append_weather_data(json)
        
create_report()

{'name': 'Glasgow', 'country': 'United Kingdom', 'region': 'Glasgow City', 'lat': '55.862', 'lon': '-4.247', 'timezone_id': 'Europe/London', 'localtime': '2020-01-07 21:35', 'localtime_epoch': 1578432900, 'utc_offset': '0.0'}
{'name': 'Glasgow', 'country': 'United Kingdom', 'region': 'Glasgow City', 'lat': '55.862', 'lon': '-4.247', 'timezone_id': 'Europe/London', 'localtime': '2020-01-07 21:35', 'localtime_epoch': 1578432900, 'utc_offset': '0.0'}
{'name': 'Glasgow', 'country': 'United Kingdom', 'region': 'Glasgow City', 'lat': '55.862', 'lon': '-4.247', 'timezone_id': 'Europe/London', 'localtime': '2020-01-07 21:35', 'localtime_epoch': 1578432900, 'utc_offset': '0.0'}
{'name': 'Leeming', 'country': 'United Kingdom', 'region': 'West Yorkshire', 'lat': '53.807', 'lon': '-1.933', 'timezone_id': 'Europe/London', 'localtime': '2020-01-07 21:35', 'localtime_epoch': 1578432900, 'utc_offset': '0.0'}
{'name': 'Leeming', 'country': 'United Kingdom', 'region': 'West Yorkshire', 'lat': '53.807', 